In [ ]:
import requests
import json
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
#Con este script comenzamos el desarrollo para llegar a la api:
#lat=31&lon=64
if __name__ == '__main__':
  url = 'https://api.openweathermap.org/data/2.5/weather?lat=31&lon=64&appid=cc3d07bf7909e147cd7443c0415b0f76&units=metric'

  response = requests.get(url)
  if response.status_code == 200:
    response_json = response.json()
    with open("Resistencia_weather.json", 'w') as outfile:
      json.dump(response_json, outfile)

    print(response_json)


{'coord': {'lon': 64, 'lat': 31}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 29.96, 'feels_like': 28.27, 'temp_min': 29.96, 'temp_max': 29.96, 'pressure': 997, 'humidity': 22, 'sea_level': 997, 'grnd_level': 919}, 'visibility': 10000, 'wind': {'speed': 4.29, 'deg': 191, 'gust': 5.11}, 'clouds': {'all': 0}, 'dt': 1690500977, 'sys': {'country': 'AF', 'sunrise': 1690505890, 'sunset': 1690555356}, 'timezone': 16200, 'id': 1140043, 'name': 'Helmand Province', 'cod': 200}


In [ ]:
print(type(response_json))

<class 'dict'>


In [ ]:
#Acá quisimos cambiar la fecha que mostraba en unix y transformarla no solo a datetime sino también cambiarla a la zona horaria de argentina.

unix_timestamp = response_json['dt']

# Convertir el tiempo Unix a una fecha y hora en UTC
utc_datetime = datetime.utcfromtimestamp(unix_timestamp)

gmt3_offset = timedelta(hours=-3)
argentina_datetime = utc_datetime + gmt3_offset

In [ ]:
print(argentina_datetime)

2023-07-20 20:36:57


In [ ]:
#Aquí simplemente buscabamos ver que tipo de dato contiene la response:
if __name__ == '__main__':
  url = 'https://api.openweathermap.org/data/2.5/weather?lat=-27.451100&lon=-58.986600&appid=cc3d07bf7909e147cd7443c0415b0f76&units=metric'
  response = requests.get(url)
  if response.status_code == 200:
    response_json = response.json()
    print(type(response_json))

<class 'dict'>


In [ ]:
"sys":{
      "type":1,
      "id":8257,
      "country":"AR",
      "sunrise":1689849797,
      "sunset":1689888055
   }

In [ ]:
# Tomar las claves 'dt', 'sunrise' y 'sunset' del JSON de respuesta
dt_value = response_json['dt']
sunrise_value = response_json['sys']['sunrise']
sunset_value = response_json['sys']['sunset']

# Crear un DataFrame con los datos
data = {'dt': [dt_value], 'sunrise': [sunrise_value], 'sunset': [sunset_value]}
df = pd.DataFrame(data)

In [ ]:
print(type(dt_value))

<class 'int'>


In [ ]:
df

,dt,sunrise,sunset
0,1690325183,1690281667,1690320210


In [ ]:
#Acá mostramos como con lambda x podemos extraer esas 3 columnas y cambiarlas al formato de datetime.
datecols = ['dt', 'sunrise', 'sunset']
df[datecols] = df[datecols].apply(lambda x: pd.to_datetime(x, unit='s'))
print(df)

                   dt             sunrise              sunset
0 2023-07-25 22:46:23 2023-07-25 10:41:07 2023-07-25 21:23:30


In [ ]:
pip install --upgrade pandas

In [ ]:
from pandas import json_normalize

# Normalizar el JSON de respuesta utilizando json_normalize
normalized_data = json_normalize(response_json)

print(normalized_data)

                                             weather      base  visibility  \
0  [{'id': 804, 'main': 'Clouds', 'description': ...  stations       10000   

           dt  timezone       id         name  cod  coord.lon  coord.lat  ...  \
0  1690325183    -10800  3429577  Resistencia  200   -58.9866   -27.4511  ...   

   main.pressure  main.humidity  wind.speed  wind.deg  clouds.all  sys.type  \
0           1013             90        1.03       180         100         1   

   sys.id  sys.country  sys.sunrise  sys.sunset  
0    8257           AR   1690281667  1690320210  

[1 rows x 24 columns]


In [ ]:
normalized_data

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1690325183,-10800,3429577,Resistencia,200,-58.9866,-27.4511,...,1013,90,1.03,180,100,1,8257,AR,1690281667,1690320210


In [ ]:
weather_data = response_json['weather']

In [ ]:
weather_data

[{'id': 804,
  'main': 'Clouds',
  'description': 'overcast clouds',
  'icon': '04n'}]

In [ ]:
normalized_weather = json_normalize(weather_data)

In [ ]:
normalized_weather

,id,main,description,icon
0,804,Clouds,overcast clouds,04n


In [ ]:
result_df = pd.concat([df, normalized_weather], axis=1)

In [ ]:
result_df

,dt,sunrise,sunset,id,main,description,icon
0,2023-07-25 22:46:23,2023-07-25 10:41:07,2023-07-25 21:23:30,804,Clouds,overcast clouds,04n


In [ ]:
# Eliminar la columna 'weather' del JSON original
del response_json['weather']

In [ ]:
# Lista de claves a eliminar
keys_to_remove = ['weather', 'base', 'timezone', 'id', 'name', 'cod']

# Eliminar las columnas del JSON original
for key in keys_to_remove:
    del response_json[key]

In [ ]:
# https://www.notion.so/campiasjuan/Modulos-importantes-41201310441b4342838fd6434bd0d742  documentacion datetime
from datetime import datetime, timedelta

fecha_actual = datetime.datetime.now().strftime("%Y%m%d")
with open(f"data_analytics\openweather\tiempodiario_{fecha_actual}.csv", 'w') as output_file:
      df.to_csv(output_file, index=False)

print(f"El DataFrame ha sido guardado en {output_file}.")

In [ ]:
import config

BASE_URL = 'https://api.openweathermap.org/data/2.5/weather?'
cityList = ["London", "New York", "Cordoba", "Taipei", "Buenos Aires", "Mexico DF", "Dublin", "Tilfis", "Bogota", "Tokio"]
coordList = ["lat=31&lon=64", "lat=40&lon=-73", "lat=-31&lon=-64", "lat=25&lon=64", "lat=-34&lon=-58", "lat=19&lon=-99", "lat=53&lon=6", "lat=41&lon=44", "lat=4&lon=74", "lat=35&lon=139"]
datos_climaticos = []

lond = ['lat=-27.451100&lon=-58.986600']

token = f'&appid={config.token1}'
https://api.openweathermap.org/data/2.5/weather?lat=53&lon=6&dt=1690500403&appid=cc3d07bf7909e147cd7443c0415b0f76&units=metric
https://api.openweathermap.org/data/2.5/weather?lat=31&lon=64&dt={int(ts)}&appid=cc3d07bf7909e147cd7443c0415b0f76&units=metric
endpoint_completo='{BASE_URL}{coordList[i]}&dt={int(ts)}&appid=cc3d07bf7909e147cd7443c0415b0f76&units=metric'

In [ ]:
for j in range(5):
  url = f"{BASE_URL}{coordList[i]}&dt={int(ts)}&appid=cc3d07bf7909e147cd7443c0415b0f76&units=metric"
  timestamp = ts
  #Convertir date time a unix timestamp para que openweather pueda leer la fecha y nosotros restar o sumar días(86400)
  dt = datetime.
  date = dt.strftime("%Y-%m-%d")
  ts = ts + 86400 #aca ya tiene un día más
  response = requests.get(url).json()
  datos_climaticos.append(response)
  with open(f"{cityList[i]}_{date}_weather.json", 'w') as outfile:
    json.dump(response, outfile, indent=4, separators=(',',': '))

In [ ]:
cityList = ["London", "New York", "Cordoba", "Taipei", "Buenos Aires", "Mexico DF", "Dublin", "Tilfis", "Bogota", "Tokio"]
coordList = ["lat=31&lon=64", "lat=40&lon=-73", "lat=-31&lon=-64", "lat=25&lon=64", "lat=-34&lon=-58", "lat=19&lon=-99", "lat=53&lon=6", "lat=41&lon=44", "lat=4&lon=74", "lat=35&lon=139"]

In [ ]:
print(len(cityList))

10


In [ ]:
x = range(len(cityList))
for n in x:
  print(n)

0
1
2
3
4
5
6
7
8
9


In [ ]:
for i in range(len(cityList)):

In [ ]:
datetime.now()